## Building The Model
-----

### Setup:
---

In [2]:
# Import dependencies
from pathlib import Path
import pandas as pd

In [4]:
# Import the Combined Data from /Wrangling_Data_Exports
wrangled_df = pd.read_csv("Wrangling_Data_Exports/Wrangled_Data.csv")

# Display Dataframe
wrangled_df.head()

,Unnamed: 0,Year,Month,Day,Time (EST),Season Week,Venue,City,State,Weather Condition,...,Home Team Pre-Game Season W/L Ratio,Home Team Pre-Game Season Avg Points For,Home Team Pre-Game Season Avg Points Against,Outcome,Away Team,Away Team Pre-Game Season W Streak,Away Team Pre-Game Season L Streak,Away Team Pre-Game Season W/L Ratio,Away Team Pre-Game Season Avg Points For,Away Team Pre-Game Season Avg Points Against
0,0,2022.0,September,25,14:25,3.0,State Farm Stadium,Glendale,Arizona,None (retractable roof closed),...,0.50,25.00,33.50,2.0,Los Angeles Rams,1.0,0.0,0.50,20.50,29.00
1,1,2022.0,September,25,14:25,3.0,State Farm Stadium,Glendale,Arizona,None (retractable roof closed),...,0.50,25.00,33.50,2.0,Los Angeles Rams,1.0,0.0,0.50,20.50,29.00
2,2,2022.0,November,6,16:05,9.0,State Farm Stadium,Glendale,Arizona,Sunny,...,0.38,22.75,26.25,2.0,Seattle Seahawks,3.0,0.0,0.62,26.25,24.88
3,3,2022.0,November,6,16:05,9.0,State Farm Stadium,Glendale,Arizona,Sunny,...,0.38,22.75,26.25,2.0,Seattle Seahawks,3.0,0.0,0.62,26.25,24.88
4,4,2022.0,November,27,16:05,12.0,State Farm Stadium,Glendale,Arizona,None (retractable roof closed),...,0.36,21.82,26.91,2.0,Los Angeles Chargers,0.0,2.0,0.50,22.70,25.80


### Prepare the Data:
---

In [ ]:
# Git Dummies?

In [ ]:
# Primary Component Analysis?

In [ ]:
# Scale the Data

In [ ]:
# Train/Test Split The Data

### Prepare the Model:
---

In [ ]:
# Create The Model/Models

### Train the Model:
---

In [ ]:
# Train the Model/Models

### Test the Model:
---

In [ ]:
# Test the Model/Models

### Analyze the Accuracy of the Model:
---

### Export the Trained Model:
---

In [ ]:
# Export the Trained Model to ../04-Creating_Applicaton/Building_Model_Exports
